In [15]:
import glob
import os
import datetime
from random import sample

from ipywidgets import IntProgress
from IPython.display import display, HTML

import girder_client
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import torch
from pytorch_unet import UNet
import evaluation_metrics
import tensorflow as tf

In [2]:
this_notebook_name = "PyTorchSagittalSpineSegmentationTest"

# Update this folder name for your computer

train_timestamp = "2021-07-28_23-42-59"
local_data_folder = r"/home/nick/dev/SaggitalSpineSegmentation_Data"

overwrite_existing_data_files = False

# All results and output will be archived with this timestamp

# Evaluation parameters

acceptable_margin_mm = 1.0
mm_per_pixel = 1.0

roc_thresholds = [0.9, 0.8, 0.7, 0.65, 0.6, 0.55, 0.5, 0.45, 0.4, 0.35, 0.3, 0.25, 0.2, 0.15, 0.1,
                  0.08, 0.06, 0.04, 0.02, 0.01,
                  0.008, 0.006, 0.004, 0.002, 0.001]

limit_rounds = 0

# Uncomment for faster debugging
# roc_thresholds = [0.8, 0.6, 0.4, 0.2, 0.1, 0.01]
# limit_rounds = 2

In [3]:
# # Use cuda GPU if available

# device_name = " "
# if torch.cuda.is_available():
#     device_name = torch.cuda.get_device_name(torch.cuda.current_device())
# else:
#     device_name = 'CPU'
    
# print('Using device:', device_name)

# device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
testing_ultrasound_filename = "ultrasound-test.npy"
testing_ultrasound_id = "5daa85edd9e6a3be02d012e7"
testing_segmentation_filename = "segmentation-test.npy"
testing_segmentation_id = "5daa85e7d9e6a3be02d012e4"

# Default subfolders of main project data folder

data_arrays_folder      = "DataArrays"
notebooks_save_folder   = "SavedNotebooks"
models_folder           = "SavedModels"
results_save_folder     = "SavedResults"
test_predictions_folder = "PredictionsTest"

data_arrays_fullpath      = os.path.join(local_data_folder, data_arrays_folder)
notebooks_save_fullpath   = os.path.join(local_data_folder, notebooks_save_folder)
models_fullpath           = os.path.join(local_data_folder, models_folder)
results_save_fullpath     = os.path.join(local_data_folder, results_save_folder)
test_predictions_fullpath = os.path.join(local_data_folder, test_predictions_folder)

if not os.path.exists(data_arrays_fullpath):
    os.makedirs(data_arrays_fullpath)
    print("Created folder: {}".format(data_arrays_fullpath))

if not os.path.exists(notebooks_save_fullpath):
    os.makedirs(notebooks_save_fullpath)
    print("Created folder: {}".format(notebooks_save_fullpath))

if not os.path.exists(models_fullpath):
    raise FileNotFoundError(models_fullpath)

if not os.path.exists(results_save_fullpath):
    os.makedirs(results_save_fullpath)
    print("Created folder: {}".format(results_save_fullpath))

if not os.path.exists(test_predictions_fullpath):
    os.makedirs(test_predictions_fullpath)
    print("Created folder: {}".format(test_predictions_fullpath))

In [5]:
# Download data from Girder

time_download_start = datetime.datetime.now()

print("Downloading testing files ...")

# Preparing progress bar

girder_api_url = "https://pocus.cs.queensu.ca/api/v1"
gclient = girder_client.GirderClient(apiUrl=girder_api_url)

test_ultrasound_fullname = os.path.join(data_arrays_fullpath, testing_ultrasound_filename)
if not os.path.exists(test_ultrasound_fullname):
    print("Downloading {}...".format(test_ultrasound_fullname))
    gclient.downloadFile(testing_ultrasound_id, test_ultrasound_fullname)

test_segmentation_fullname = os.path.join(data_arrays_fullpath, testing_segmentation_filename)
if not os.path.exists(test_segmentation_fullname) or overwrite_existing_data_files:
    print("Downloading {}...".format(test_segmentation_fullname))
    gclient.downloadFile(testing_segmentation_id, test_segmentation_fullname)
    
time_download_stop = datetime.datetime.now()
print("\nTotal download time: {}".format(time_download_stop - time_download_start))


Total download time: 0:00:00.000903


In [21]:
# Load numpy data
# convert ultrasound array to torch.tensor channels first
# Keep segmentation in numpy/TF format

time_start = datetime.datetime.now()

test_ultrasound_fullname = os.path.join(data_arrays_fullpath, testing_ultrasound_filename)
test_ultrasound_array = np.load(test_ultrasound_fullname)
test_ultrasound_array = torch.Tensor(test_ultrasound_array).permute(0,3,1,2).float()

test_segmentation_fullname = os.path.join(data_arrays_fullpath, testing_segmentation_filename)
test_segmentation_array = np.load(test_segmentation_fullname)

time_stop = datetime.datetime.now()
print("\nTotal time to load from files: {}".format(time_stop - time_start))


Total time to load from files: 0:00:00.035368


In [24]:
print(test_ultrasound_array.size(), test_segmentation_array.shape)

torch.Size([1892, 1, 128, 128]) (1892, 128, 128, 1)


In [8]:
filename_pattern = "*" + train_timestamp + "*"
search_string = os.path.join(models_fullpath, filename_pattern)
print("Searching for models by {}".format(search_string))
model_file_list = glob.glob(search_string)

num_models = len(model_file_list)
print("Found {} models".format(num_models))

if limit_rounds > 0:
    num_rounds = min(num_models, limit_rounds)
else:
    num_rounds = num_models

Searching for models by /home/nick/dev/SaggitalSpineSegmentation_Data/SavedModels/*2021-07-28_23-42-59*
Found 8 models


In [9]:
# wcce = torch.nn.CrossEntropyLoss(weight=torch.tensor([0.05, 0.95]).float())
# wcce = weighted_categorical_crossentropy([0.05, 0.95])

In [10]:
print(model_file_list)

['/home/nick/dev/SaggitalSpineSegmentation_Data/SavedModels/PyTorchSagittalSpineSegmentationStudy_model-3_2021-07-28_23-42-59.msd', '/home/nick/dev/SaggitalSpineSegmentation_Data/SavedModels/PyTorchSagittalSpineSegmentationStudy_model-4_2021-07-28_23-42-59.msd', '/home/nick/dev/SaggitalSpineSegmentation_Data/SavedModels/PyTorchSagittalSpineSegmentationStudy_model-7_2021-07-28_23-42-59.msd', '/home/nick/dev/SaggitalSpineSegmentation_Data/SavedModels/PyTorchSagittalSpineSegmentationStudy_model-0_2021-07-28_23-42-59.msd', '/home/nick/dev/SaggitalSpineSegmentation_Data/SavedModels/PyTorchSagittalSpineSegmentationStudy_model-5_2021-07-28_23-42-59.msd', '/home/nick/dev/SaggitalSpineSegmentation_Data/SavedModels/PyTorchSagittalSpineSegmentationStudy_model-1_2021-07-28_23-42-59.msd', '/home/nick/dev/SaggitalSpineSegmentation_Data/SavedModels/PyTorchSagittalSpineSegmentationStudy_model-2_2021-07-28_23-42-59.msd', '/home/nick/dev/SaggitalSpineSegmentation_Data/SavedModels/PyTorchSagittalSpineSeg

In [25]:
# Main loop, test all models and save test results

time_sequence_start = datetime.datetime.now()

print("Using timestemp to find files: {}".format(train_timestamp))
print("Saving test predictions in:    {}".format(test_predictions_fullpath))

test_best_metrics    = dict()
test_fuzzy_metrics   = dict()
test_aurocs          = np.zeros(num_models)
test_best_thresholds = np.zeros(num_models)
test_prediction_time = np.zeros(num_models)

for i in range(num_models):
    time_round_start = datetime.datetime.now()
    
    model = UNet(128,2).eval()
    
    print("Testing model: {}".format(model_file_list[i]))
    model.load_state_dict(torch.load(model_file_list[i]))
    test_prediction = model(test_ultrasound_array)
    test_prediction = torch.nn.Softmax(dim=1)(test_prediction)
    test_prediction = test_prediction.permute(0,2,3,1)
    test_prediction = np.array(test_prediction.detach().numpy())
    print(np.shape(test_prediction))
    
    test_prediction_filename = train_timestamp + "_prediction_test.npy"
    test_prediction_fullname = os.path.join(test_predictions_fullpath, test_prediction_filename)
    np.save(test_prediction_fullname, test_prediction)
    
    # Test results
    
    test_metrics_dicts, test_best_threshold_index, test_area = evaluation_metrics.compute_roc(
        roc_thresholds, test_prediction, test_segmentation_array, acceptable_margin_mm, mm_per_pixel)
    
    test_fuzzy_metrics[i] = evaluation_metrics.compute_evaluation_metrics(
        test_prediction, test_segmentation_array, acceptable_margin_mm, mm_per_pixel)
    
    test_best_metrics[i]    = test_metrics_dicts[test_best_threshold_index]
    test_aurocs[i]          = test_area
    test_best_thresholds[i] = roc_thresholds[test_best_threshold_index]
    
    print("Testing round time: {}".format(datetime.datetime.now() - time_round_start))


time_sequence_stop = datetime.datetime.now()

print("\nTotal testing time:   {}".format(time_sequence_stop - time_sequence_start))

Using timestemp to find files: 2021-07-28_23-42-59
Saving test predictions in:    /home/nick/dev/SaggitalSpineSegmentation_Data/PredictionsTest
Testing model: /home/nick/dev/SaggitalSpineSegmentation_Data/SavedModels/PyTorchSagittalSpineSegmentationStudy_model-3_2021-07-28_23-42-59.msd
(1892, 128, 128, 2)
Testing round time: 0:00:42.933560
Testing model: /home/nick/dev/SaggitalSpineSegmentation_Data/SavedModels/PyTorchSagittalSpineSegmentationStudy_model-4_2021-07-28_23-42-59.msd
(1892, 128, 128, 2)
Testing round time: 0:00:42.899110
Testing model: /home/nick/dev/SaggitalSpineSegmentation_Data/SavedModels/PyTorchSagittalSpineSegmentationStudy_model-7_2021-07-28_23-42-59.msd
(1892, 128, 128, 2)
Testing round time: 0:00:43.210762
Testing model: /home/nick/dev/SaggitalSpineSegmentation_Data/SavedModels/PyTorchSagittalSpineSegmentationStudy_model-0_2021-07-28_23-42-59.msd
(1892, 128, 128, 2)
Testing round time: 0:00:43.913363
Testing model: /home/nick/dev/SaggitalSpineSegmentation_Data/Sav

In [26]:
# Arrange results in tables

metric_labels = [
    "AUROC",
    "best thresh",
    "best TP",
    "best FP",
    "best recall",
    "best precis",
    "fuzzy recall",
    "fuzzy precis",
    "fuzzy Fscore"
]

results_labels = []

for label in metric_labels:
    results_labels.append("Test " + label)

    
results_df = pd.DataFrame(columns = results_labels)

for i in range(num_rounds):
    results_df.loc[i] = [
        test_aurocs[i],
        test_best_thresholds[i],
        test_best_metrics[i][evaluation_metrics.TRUE_POSITIVE_RATE],
        test_best_metrics[i][evaluation_metrics.FALSE_POSITIVE_RATE],
        test_best_metrics[i][evaluation_metrics.RECALL],
        test_best_metrics[i][evaluation_metrics.PRECISION],
        test_fuzzy_metrics[i][evaluation_metrics.RECALL],
        test_fuzzy_metrics[i][evaluation_metrics.PRECISION],
        test_fuzzy_metrics[i][evaluation_metrics.FSCORE],
    ]

display(results_df)

print("\nAverages")

results_means_df = results_df.mean()
display(results_means_df)

,Test AUROC,Test best thresh,Test best TP,Test best FP,Test best recall,Test best precis,Test fuzzy recall,Test fuzzy precis,Test fuzzy Fscore
0,0.880390,0.010,0.929612,0.232987,0.929612,0.010971,0.058482,0.013502,0.021939
1,0.873368,0.020,0.879428,0.187280,0.879428,0.012885,0.052202,0.012385,0.020020
2,0.890357,0.020,0.908031,0.190649,0.908031,0.013067,0.064450,0.013880,0.022841
3,0.875339,0.010,0.922251,0.233923,0.922251,0.010842,0.052323,0.013162,0.021033
4,0.883948,0.010,0.942666,0.245236,0.942666,0.010574,0.056003,0.014845,0.023468
5,0.846527,0.006,0.850429,0.228931,0.850429,0.010220,0.041621,0.011437,0.017943
6,0.860039,0.010,0.901942,0.231736,0.901942,0.010704,0.053601,0.006347,0.011350
7,0.862573,0.008,0.881307,0.209639,0.881307,0.011551,0.035447,0.012941,0.018960



Averages


Test AUROC           0.871568
Test best thresh     0.011750
Test best TP         0.901958
Test best FP         0.220048
Test best recall     0.901958
Test best precis     0.011352
Test fuzzy recall    0.051766
Test fuzzy precis    0.012312
Test fuzzy Fscore    0.019694
dtype: float64

In [ ]:
# Save results table

csv_filename = this_notebook_name + "_" + train_timestamp + ".csv"
csv_fullname = os.path.join(results_save_fullpath, csv_filename)
results_df.to_csv(csv_fullname)

print("Results saved to: {}".format(csv_fullname))

In [ ]:
# print(float(str(sf_time)[5:]))

In [ ]:
# GET SINGLE-FRAME TIMINGS

num_test = test_ultrasound_array.shape[0]
num_show = 4

indices = [i for i in range(num_test)]
sample_indices = sample(indices, num_show)

model = load_model(model_file_list[1], custom_objects={'loss': wcce, 'dice_coef': dice_coef})
# test_prediction_singleframe = model.predict(test_ultrasound_array[sample_indices[0]][np.newaxis, :, :, :])

fig = plt.figure(figsize=(18, num_show*5))
for i in range(num_show):
    sf_start = datetime.datetime.now()
    test_prediction_singleframe = model.predict(test_ultrasound_array[sample_indices[i]][np.newaxis, :, :, :])
    test_prediction_singleframe = np.array(test_prediction_singleframe)
    b = np.sum(test_prediction_singleframe)
    sf_end = datetime.datetime.now()
    
    sf_time = sf_end - sf_start
#     print(sf_time.dtype)
#     print(np.shape(test_prediction_singleframe))
    
    a0 = fig.add_subplot(num_show,3,i*3+1)
    img0 = a0.imshow(test_ultrasound_array[sample_indices[i], :, :, 0].astype(np.float32))
    a0.set_title("Ultrasound #{}".format(sample_indices[i]), fontsize=18)
    a1 = fig.add_subplot(num_show,3,i*3+2)
    img1 = a1.imshow(test_segmentation_array[sample_indices[i], :, :, 0], vmin=0.0, vmax=1.0)
    a1.set_title("Segmentation #{}".format(sample_indices[i]), fontsize=18)
    c = fig.colorbar(img1, fraction=0.046, pad=0.04)
    a2 = fig.add_subplot(num_show,3,i*3+3)
    img2 = a2.imshow(test_prediction_singleframe[0, :, :, 1], vmin=0.0, vmax=1.0)
    a2.set_title("Prediction #{} (".format(sample_indices[i]) + str(sf_time)[5:] + " seconds)", fontsize=18)
    c = fig.colorbar(img2, fraction=0.046, pad=0.04)
    
fig.suptitle("U-Net Single-frame Prediction Timings on Saggital Spine Data\n", fontsize=30)
fig.tight_layout()

In [ ]:
# GET MULTI-FRAME TIMINGS

tabl = PrettyTable()


metric_labels = [
    "Frames",
    "# Samples",
    "First PT (s)",
    "Avg PT (s)",
    "Min PT (s)",
    "Max PT (s)",
    "Std. Dev. (s)",
]

tabl.field_names = metric_labels
results_df = pd.DataFrame(columns = metric_labels)

# Main Loop

frames = [1, 10, 20, 30, 40, 50]
num_test = test_ultrasound_array.shape[0]

for l in range(len(frames)):

    indices = [i for i in range(num_test - frames[l])]

    model = load_model(model_file_list[1], custom_objects={'loss': wcce, 'dice_coef': dice_coef})
    mf_times = []

    for i in indices:
        mf_start = datetime.datetime.now()
        test_prediction_singleframe = model.predict(test_ultrasound_array[i:i + frames[l], :, :, :])
        test_prediction_singleframe = np.array(test_prediction_singleframe)
        b = np.sum(test_prediction_singleframe)
        mf_end = datetime.datetime.now()

        mf_time = mf_end - mf_start
        mf_time_seconds = float(str(mf_time)[5:])
        mf_times.append(mf_time_seconds)
    
#     tabl.add_row([
#         frames[l],
#         num_test - frames[l],
#         mf_times[0],
#         np.mean(mf_times),
#         np.min(mf_times),
#         np.max(mf_times),
#         np.std(mf_times)
#     ])
    
    results_df.loc[l] = [
        frames[l],
        num_test - frames[l],
        mf_times[0],
        int((np.mean(mf_times)*100000)) / 100000,
        np.min(mf_times),
        np.max(mf_times),
        int((np.std(mf_times)*100000)) / 100000
    ]

# print(tabl)
print("Multi-Frame \"Sliding Window\" Prediction Times on Saggital Spine Data")
print("PT = Prediction Time")
display(results_df)

In [ ]:
# GET MULTI-FRAME TIMINGS WITH WARM-UP

tabl = PrettyTable()


metric_labels = [
    "Frames",
    "# Samples",
    "First PT (s)",
    "Avg PT (s)",
    "Min PT (s)",
    "Max PT (s)",
    "Std. Dev. (s)",
]

tabl.field_names = metric_labels
results_df = pd.DataFrame(columns = metric_labels)

# Main Loop

frames = [1, 10, 20, 30, 40, 50]
num_test = test_ultrasound_array.shape[0]

for l in range(len(frames)):

    indices = [i for i in range(num_test - frames[l])]

    model = load_model(model_file_list[1], custom_objects={'loss': wcce, 'dice_coef': dice_coef})
    test_prediction_singleframe = model.predict(test_ultrasound_array[0:0 + frames[l], :, :, :])
    mf_times = []

    for i in indices:
        mf_start = datetime.datetime.now()
        test_prediction_singleframe = model.predict(test_ultrasound_array[i:i + frames[l], :, :, :])
        test_prediction_singleframe = np.array(test_prediction_singleframe)
        b = np.sum(test_prediction_singleframe)
        mf_end = datetime.datetime.now()

        mf_time = mf_end - mf_start
        mf_time_seconds = float(str(mf_time)[5:])
        mf_times.append(mf_time_seconds)
    
#     tabl.add_row([
#         frames[l],
#         num_test - frames[l],
#         mf_times[0],
#         np.mean(mf_times),
#         np.min(mf_times),
#         np.max(mf_times),
#         np.std(mf_times)
#     ])
    
    results_df.loc[l] = [
        frames[l],
        num_test - frames[l],
        mf_times[0],
        int((np.mean(mf_times)*100000)) / 100000,
        np.min(mf_times),
        np.max(mf_times),
        int((np.std(mf_times)*100000)) / 100000
    ]

# print(tabl)
print("Multi-Frame \"Sliding Window\" Prediction Times with \"warm-up\" on Saggital Spine Data")
print("PT = Prediction Time")
display(results_df)

In [ ]:
# Save notebook so all output is archived by the next cell

from IPython.display import Javascript
script = '''
require(["base/js/namespace"],function(Jupyter) {
    Jupyter.notebook.save_checkpoint();
});
'''
Javascript(script)

In [ ]:
# Export HTML copy of this notebook

notebook_file_name = this_notebook_name + "_" + train_timestamp + ".html"
notebook_fullname = os.path.join(notebooks_save_fullpath, notebook_file_name)

os.system("jupyter nbconvert --to html " + this_notebook_name + " --output " + notebook_fullname)
print("Notebook saved to: {}".format(notebook_fullname))